# Agents with examples

In [ ]:
# %uv pip install agentor

## What is an Agent?

<img src="https://i.imgflip.com/a7rr5a.jpg" width="250px">

<br>

## Agents are LLMs with higher autonomy

- Tool access
- Memory

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from openai import OpenAI

client = OpenAI()
response = client.responses.create(
    model="gpt-5-nano", input="Write a haiku about recursion in programming."
)

print(response.output[-1].content[0].text)

In [ ]:
from agents import Agent, Runner

load_dotenv()

agent = Agent(
    name="Assistant", instructions="You are a helpful assistant", model="gpt-5-nano"
)

result = await Runner.run(agent, "Write a haiku about recursion in programming.")
print(result.final_output)

Code calls back to code
Until it reaches the base
Return to the start


In [ ]:
agent = Agent(name="Assistant", instructions="Reply very concisely.")

result = await Runner.run(agent, "What city is the Golden Gate Bridge in?")
print(result.final_output)

San Francisco.


# Tool use

In [ ]:
result = await Runner.run(agent, "What is the weather in London?")
print(result.final_output)

I can't access real-time data. Please check a weather website or app for current London weather.


In [ ]:
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

Use Celesto Tool Hub for a realtime access to weather data and 100+ tools.

```python
from agentor import CelestoSDK

client = CelestoSDK(CELESTO_API_KEY)

# List all available tools
client.toolhub.list_tools()

# Run the weather tool for a specific location
client.toolhub.run_weather_tool("London")
```

In [ ]:
import os

from agentor import CelestoSDK

client = CelestoSDK(os.environ.get("CELESTO_API_KEY"))

# List all available tools
client.toolhub.list_tools()

# Run the weather tool for a specific location
client.toolhub.run_weather_tool("London")

/Users/aniket/Projects/celesto/agentor/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'tool_id': None,
 'tool_name': 'current-weather',
 'output': {'location': {'name': 'London',
   'region': 'City of London, Greater London',
   'country': 'United Kingdom',
   'lat': 51.5171,
   'lon': -0.1062,
   'tz_id': 'Europe/London',
   'localtime_epoch': 1765462766,
   'localtime': '2025-12-11 14:19'},
  'current': {'last_updated_epoch': 1765462500,
   'last_updated': '2025-12-11 14:15',
   'temp_c': 12.3,
   'temp_f': 54.1,
   'is_day': 1,
   'condition': {'text': 'Partly cloudy',
    'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
    'code': 1003},
   'wind_mph': 8.9,
   'wind_kph': 14.4,
   'wind_degree': 176,
   'wind_dir': 'S',
   'pressure_mb': 1017.0,
   'pressure_in': 30.03,
   'precip_mm': 0.0,
   'precip_in': 0.0,
   'humidity': 77,
   'cloud': 50,
   'feelslike_c': 10.8,
   'feelslike_f': 51.5,
   'windchill_c': 9.0,
   'windchill_f': 48.1,
   'heatindex_c': 10.8,
   'heatindex_f': 51.4,
   'dewpoint_c': 7.1,
   'dewpoint_f': 44.7,
   'vis_km': 10.0,
   'vi

In [ ]:
from agentor import Agentor, tool


@tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"


agent = Agentor(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-nano",
    tools=[get_weather],
)

In [ ]:
result = await agent.arun("What is the weather in London?")

In [ ]:
print(result.final_output)

No live weather here
London skies drift, rain or sun
check a weather app


## LLM with tool use

In [ ]:
weather_tool_def = {
    "type": "function",
    "name": "get_weather",
    "description": "Retrieves current weather for the given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. London, United Kingdom",
            },
            "units": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "Units the temperature will be returned in.",
            },
        },
        "required": ["location", "units"],
        "additionalProperties": False,
    },
    "strict": True,
}


def get_weather(location: str, units: str) -> str:
    """returns weather info for the specified location and units."""
    return (
        f"The weather in {location} is sunny with a temperature of 20 degrees {units}."
    )


tool_registry = {"get_weather": get_weather}

In [ ]:
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano", input="What is the weather in London?", tools=[weather_tool_def]
)

In [ ]:
print(response.output[-1])

ResponseFunctionToolCall(arguments='{"location":"London, United Kingdom","units":"celsius"}', call_id='call_AqNBXfEQrt2HT1nU2IFO8Zhc', name='get_weather', type='function_call', id='fc_09b197958f1909bb00693ad41887f88193b9626df0b8fbeb58', status='completed')


In [ ]:
import json

if response.output[-1].type == "function_call":
    fn_name = response.output[-1].name
    arguments = json.loads(response.output[-1].arguments)
    result = tool_registry[fn_name](**arguments)
    print(result)
else:
    print(response.output[-1].content[0].text)

The weather in London, United Kingdom is sunny with a temperature of 20 degrees celsius.
